<div style="border-radius: 10px; border: #DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size: 100%; text-align: left">
<h3 align="left"><font color='#3498DB'>Exploratory Analysis of Bayesian Models</font></h3>

* コードは公開されている以下のオンラインブックから引用しています。
* Martin Osvaldo A, Kumar Ravin; Lao Junpeng Bayesian Modeling and Computation in Python Boca Ratón, 2021. ISBN 978-0-367-89436-8
* https://bayesiancomputationbook.com/welcome.html

In [9]:
%matplotlib inline
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
from scipy import stats

In [10]:
az.style.use("arviz-grayscale")
plt.rcParams['figure.dpi'] = 300
np.random.seed(5201)

In [13]:
half_length = 3.66
penalty_point = 11
def Phi(x):
    """標準正規分布の分布関数を計算する"""
    return 0.5 + 0.5 * pm.math.erf(x / (2.0)**0.5)

ppss = []
sigmas_deg = [5, 20, 60]
sigmas_rad = np.deg2rad(sigmas_deg)
for sigma in sigmas_rad:
    with pm.Model() as model:
        σ = pm.HalfNormal("σ", sigma)
        α = pm.Normal("α", 0, σ)
        p_goal = pm.Deterministic("p_goal", 2 * Phi(pm.math.arctan(half_length/ penalty_point)/σ)-1)
        pps = pm.sample_prior_predictive(250)
        ppss.append(pps)
        